In [1]:
import pdfplumber
import os
import re
import pandas as pd
import numpy as np

In [2]:
pl_text = [  '四半期連結損益及び包括利益計算書'
           , '四半期連結損益計算書'
           , '四半期連結損益及び包括利益計算書'
           , '連結損益及び包括利益計算書'
           , '連結損益計算書']

bs_text = [  '四半期連結貸借対照表'
           , '連結貸借対照表'
           , '連結財政状態計算書']
 
def find_header(pdf,
                page_id: int,):
    page = pdf.pages[page_id]
    text = page.extract_text()
    DON_VI = '単位'

    idx_dv = text.find(DON_VI)
    if idx_dv >= 0:
        text1 = text[idx_dv:]
        ## Tim don vi
        donviYen = text1[3: text1.find('\n') - 1]
        ## Tim field
        text2 = text1[text1.find('\n') + 1: ]
        text3 = text2[: text2.find('\n')]
        text3 = text3.split(' ')

        if len(text3) != 2:
            return [donviYen] + ['B'] *2
        return [donviYen] + text3
    else:
        return ['N/A']*2

def check_str(text, lst_str):
    for s in lst_str:
        idx = text.find(s)
        if idx != -1:
            return idx < text.find('単位')
    return False

def extract_data_pdf(pdf,
                     lst_table = [],
                     page_id = 0,
                     type_data = 'find_keyword',
                     lst_str = []):

    page = pdf.pages[page_id]
    tables = page.extract_tables()
    text = page.extract_text()
    text = text.replace(" ","")

    if type_data == 'find_keyword':
        find_key = check_str(text, lst_str)
    if type_data == 'avoid_keyword':
        idx = text.find('\n')
        if text[idx + 2: idx + 4] == '単位':
            find_key = True
        else: find_key = False

    if ((find_key == True)
        and (len(tables) > 0)
        and (np.array(tables[0]).shape[1] >=  3)):
            for table in tables:
                for row in table:
                    row_ = [[x for x in row[:-2] if (x != None and x != '')] + row[-2:]]
                    if len(row_[0]) != 3:
                        row_[0] = ['B'] * 3
                    lst_table += row_
            return lst_table, True
    return lst_table, False

def extract_pdf(path_file, key):
    pdf = pdfplumber.open(path_file)
    for page_id in range(len(pdf.pages)):
        lst_table, _ = extract_data_pdf(pdf,
                                        lst_table = [],
                                        page_id = page_id,
                                        type_data = 'find_keyword',
                                        lst_str = key )
        if _ == True:
            lst_col = find_header(pdf, page_id = page_id)
            break

    while True:
        # print(page_id)
        lst_table, _ = extract_data_pdf(pdf,
                                        lst_table = lst_table,
                                        page_id = page_id+1,
                                        type_data = 'avoid_keyword',
                                        lst_str = [] )
        if _ == True:
            page_id += 1
        else: break
    df = pd.DataFrame(lst_table, columns = lst_col)
    return df.replace('', None)

def get_table(id_company, type_ = 'bs',
               path_save = '',
               return_df = False,
               save_file = True):
    if type_ == 'bs':
        key = bs_text
    elif type_ == 'pl':
        key = pl_text

    for file in os.listdir(path_save + f'Data/{id_company}/PDF'):
        if 'ocr' not in file and '(訂正)' not in file:
            try:
                numbers = re.findall(r'\d+', file)
                year = numbers[0]
                quy = f'Q{numbers[1]}'
                path_file = path_save + f'Data/{id_company}/PDF/{file}'
                df_ = extract_pdf(path_file, key)
                if save_file:
                    df_.to_csv(path_save + f'Data/{id_company}/docs/{type_}/{year}_{quy}.csv', index=False)
                if return_df:
                    print(f'Data/{id_company}/{type_}/{year}_{quy}')
                    print(df_.to_string())
                    print(' ')
            except:
                print(f'----------BUG---------{file}')


In [3]:
get_table(1376, type_ = 'bs',
               path_save = 'tests/',
               return_df = True,
               save_file = False)

----------BUG---------2007_Q4_決算短信(2008_7_24).pdf
Data/1376/bs/2008_Q1
              千円           B           B
0           資産の部        None        None
1           流動資産        None        None
2         現金及び預金   1,817,110   1,283,704
3      受取手形及び売掛金  16,297,155  19,459,039
4             商品   4,083,517   4,124,103
5            その他   1,167,024     742,603
6          貸倒引当金     △57,021     △68,012
7         流動資産合計  23,307,786  25,541,438
8           固定資産        None        None
9         有形固定資産        None        None
10            土地   4,086,898   4,086,898
11       その他（純額）   1,958,766   1,991,447
12        有形固定資産   6,045,665   6,078,346
13        無形固定資産     206,769     214,438
14      投資その他の資産        None        None
15           その他   2,269,482   2,360,250
16         貸倒引当金    △191,730    △181,694
17    投資その他の資産合計   2,077,751   2,178,556
18        固定資産合計   8,330,186   8,471,341
19          資産合計  31,637,972  34,012,779
20          負債の部        None        None
21          流動負債        Non

In [4]:
path_file = "tests/Data/1333/PDF/2018_Q2_決算短信(2018_11_5).pdf"
pdf = pdfplumber.open(path_file)
page = pdf.pages[5]
# text = page.extract_text()
# t = text[text.find('単位'):]
# print(t)
# convert_text_to_table(t, 3)
# check_str(text, pl_text)

# extract_data_pdf(pdf,lst_table = [], page_id = 7,
#                                         type_data = 'find_keyword',
#                                         lst_str = pl_text )
tables = page.extract_tables()
for table in tables:
        # Xử lý bảng theo nhu cầu của bạn
        # Ví dụ: in nội dung bảng
        for row in table:
            print(row)
        print("-------------")
extract_pdf(path_file, bs_text)

['', '', '（単位：百万円）']
['', '前連結会計年度\n(平成30年３月31日)', '当第２四半期連結会計期間\n(平成30年９月30日)']
['資産の部', '', '']
-------------
['現金及び預金', '16,475', '12,845']
-------------
['たな卸資産', '152,010', '166,074']
-------------
['貸倒引当金', '△373', '△396']
-------------
['固定資産', '', '']
-------------
['土地', '48,430', '48,136']
-------------
['有形固定資産合計', '138,444', '138,060']
-------------
['のれん', '9,125', '8,574']
-------------
['無形固定資産合計', '21,917', '21,165']
-------------
['投資その他の資産', '68,538', '67,353']
-------------
['投資その他の資産合計', '64,061', '62,972']
-------------
['資産合計', '516,607', '541,384']
-------------
['流動負債', '', '']
-------------
['短期借入金', '138,199', '143,051']
-------------
['その他', '43,497', '44,028']
-------------
['固定負債', '', '']
-------------
['退職給付に係る負債', '21,144', '20,819']
-------------
['その他', '7,647', '8,038']
-------------
['負債合計', '376,557', '397,378']
-------------
['株主資本', '', '']
-------------
['資本剰余金', '39,703', '39,697']
-------------
['自己株式', '△57', '△67']
-------------
['その他の包括利益累計額

,百万円,前連結会計年度,当第２四半期連結会計期間
0,B,B,B
1,B,B,B
2,資産の部,None,None
3,現金及び預金,"16,475","12,845"
4,たな卸資産,"152,010","166,074"
5,貸倒引当金,△373,△396
6,固定資産,None,None
7,土地,"48,430","48,136"
8,有形固定資産合計,"138,444","138,060"
9,のれん,"9,125","8,574"
